## ENVIRONMENT VARIABLES

In [1]:
EN_ES_CORPUS_DIR = "/notebooks/master-thesis/corpora/tatoeba-challenge/en-es/"

In [2]:
AYM_ES_CORPUS_DIR = "/notebooks/master-thesis/corpora/americasnlp2021/data/aymara-spanish"

In [3]:
tokenizer_path = "/storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/"

In [4]:
%env TOKENIZER_PATH = /storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en

env: TOKENIZER_PATH=/storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en


In [5]:
tokenized_path_es_en = "/storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/es-en/"
tokenized_path_aym_es = "/storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/aym-es/"

In [6]:
%env TOKENIZED_PATH_ES_EN = /storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/es-en
%env TOKENIZED_PATH_AYM_ES = /storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/aym-es

env: TOKENIZED_PATH_ES_EN=/storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/es-en
env: TOKENIZED_PATH_AYM_ES=/storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/aym-es


In [7]:
%env BIN_DIR = /storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/bin

env: BIN_DIR=/storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/bin


In [8]:
! mkdir -p $TOKENIZED_PATH_ES_EN
! mkdir -p $TOKENIZED_PATH_AYM_ES
! mkdir -p $BIN_DIR

In [9]:
%env MODEL_DIR = /storage/master-thesis/models/resdrop__aym-es_es-en

env: MODEL_DIR=/storage/master-thesis/models/resdrop__aym-es_es-en


In [10]:
%env CODE_STORAGE = /storage/code/

env: CODE_STORAGE=/storage/code/


In [ ]:
! locale

locale: Cannot set LC_CTYPE to default locale: No such file or directory
locale: Cannot set LC_MESSAGES to default locale: No such file or directory
locale: Cannot set LC_ALL to default locale: No such file or directory
LANG=en_US.UTF-8
LANGUAGE=
LC_CTYPE="en_US.UTF-8"
LC_NUMERIC="en_US.UTF-8"
LC_TIME="en_US.UTF-8"
LC_COLLATE="en_US.UTF-8"
LC_MONETARY="en_US.UTF-8"
LC_MESSAGES="en_US.UTF-8"
LC_PAPER="en_US.UTF-8"
LC_NAME="en_US.UTF-8"
LC_ADDRESS="en_US.UTF-8"
LC_TELEPHONE="en_US.UTF-8"
LC_MEASUREMENT="en_US.UTF-8"
LC_IDENTIFICATION="en_US.UTF-8"
LC_ALL=en_US.UTF-8


In [ ]:
! update-locale LANG=en_US.UTF-8 LANGUAGE=en.UTF-8

%env LANG=en_US.UTF-8
%env LC_CTYPE=en_US.UTF-8
%env LC_ALL=en_US.UTF-8

/bin/sh: 1: update-locale: not found
env: LANG=en_US.UTF-8
env: LC_CTYPE=en_US.UTF-8
env: LC_ALL=en_US.UTF-8


In [ ]:
! cat /etc/rc.conf

cat: /etc/rc.conf: No such file or directory


## Libraries

In [ ]:
pip install tokenizers==0.10.3

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip uninstall torch torchvision torchaudio -y

In [ ]:
pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

In [ ]:
pip uninstall fairseq -y

In [ ]:
pip install fairseq

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
! pip uninstall apex -y

In [ ]:
rm -r apex

In [ ]:
#! git clone https://github.com/NVIDIA/apex

! pip install -v --disable-pip-version-check --no-cache-dir apex

In [ ]:
pip install tensorboardX

## Converting tatoeba format to standard format

In [ ]:
import csv

with open(EN_ES_CORPUS_DIR + "original/valid.txt", encoding='utf8') as f:
    es = open(EN_ES_CORPUS_DIR + "dev.es", 'w', encoding='utf8')
    en = open(EN_ES_CORPUS_DIR + "dev.en", 'w', encoding='utf8')
    for line in csv.reader(f, delimiter="\t"):
        es.write(line[3] + "\n")
        en.write(line[2] + "\n")
    en.close()
    es.close()
    
with open(EN_ES_CORPUS_DIR + "original/dev.txt", encoding='utf8') as f:
    es = open(EN_ES_CORPUS_DIR + "train.es", 'w', encoding='utf8')
    en = open(EN_ES_CORPUS_DIR + "train.en", 'w', encoding='utf8')
    for line in csv.reader(f, delimiter="\t"):
        if "\n" in line[2] or "\n" in line[3]:
            continue
        if len(line) != 4:
            continue
            
        es.write(line[3] + "\n")
        en.write(line[2] + "\n")
        
    en.close()
    es.close()
        

### Tokenizing data

In [11]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece

en_es_aym_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

In [12]:
from tokenizers.trainers import WordPieceTrainer

en_es_aym_trainer = WordPieceTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"], show_progress=True)

In [13]:
from tokenizers.pre_tokenizers import Whitespace

en_es_aym_tokenizer.pre_tokenizer = Whitespace()

en_files = [f"/notebooks/master-thesis/corpora/tatoeba-challenge/en-es/{split}.en" for split in ["dev", "train"]]
es_files = [f"/notebooks/master-thesis/corpora/tatoeba-challenge/en-es/{split}.es" for split in ["dev", "train"]]
aym_files = [f"/notebooks/master-thesis/corpora/americasnlp2021/data/aymara-spanish/{split}.aym" for split in ["dev", "train"]]
es2_files = [f"/notebooks/master-thesis/corpora/americasnlp2021/data/aymara-spanish/{split}.es" for split in ["dev", "train"]]

files = en_files + es_files + es2_files + aym_files
#print(files)
en_es_aym_tokenizer.train(files= files, trainer=en_es_aym_trainer)

In [14]:
from pathlib import Path



def tokenize_files(tokenizer, files, extension, output_path):
    for file in files:
        print(f"Reading file {file}")
        with open(file, encoding='utf8') as f:
          lines = f.readlines()
          tokenized_lines = tokenizer.encode_batch(lines)
          tokenized_name = Path(file).stem
          tokenized_name = output_path + tokenized_name + "." + extension
          print(tokenized_name)
          with open(tokenized_name, 'w', encoding='utf8') as wf:

            wf.writelines([" ".join(t.tokens) + "\n" for t in tokenized_lines])

In [15]:
en_es_aym_tokenizer.save(tokenizer_path + "aym-es-en-tokenizer.json")

In [16]:
print(en_es_aym_tokenizer)

In [17]:
tok = en_es_aym_tokenizer.encode("we hold this truth to be self evident that everyone is created equal?")
print(tok.tokens)

['we', 'hold', 'this', 'truth', 'to', 'be', 'self', 'evident', 'that', 'everyone', 'is', 'created', 'equal', '?']


In [18]:
tokenize_files(en_es_aym_tokenizer, en_files, "en", tokenized_path_es_en)
tokenize_files(en_es_aym_tokenizer, es_files, "es", tokenized_path_es_en)

tokenize_files(en_es_aym_tokenizer, aym_files, "aym", tokenized_path_aym_es)
tokenize_files(en_es_aym_tokenizer, es2_files, "es", tokenized_path_aym_es)

Reading file /notebooks/master-thesis/corpora/tatoeba-challenge/en-es/dev.en
/storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/es-en/dev.en
Reading file /notebooks/master-thesis/corpora/tatoeba-challenge/en-es/train.en
/storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/es-en/train.en
Reading file /notebooks/master-thesis/corpora/tatoeba-challenge/en-es/dev.es
/storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/es-en/dev.es
Reading file /notebooks/master-thesis/corpora/tatoeba-challenge/en-es/train.es
/storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/es-en/train.es
Reading file /notebooks/master-thesis/corpora/americasnlp2021/data/aymara-spanish/dev.aym
/storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/aym-es/dev.aym
Reading file /notebooks/master-thesis/corpora/americasnlp2021/data/aymara-spanish/train.aym
/storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-

## Binarizing data

In [19]:
### Removing previous dict files
! rm $BIN_DIR/dict.aym.txt
! rm $BIN_DIR/dict.es.txt
! rm $BIN_DIR/dict.en.txt

rm: cannot remove '/storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/bin/dict.aym.txt': No such file or directory
rm: cannot remove '/storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/bin/dict.es.txt': No such file or directory
rm: cannot remove '/storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/bin/dict.en.txt': No such file or directory


In [20]:
## Concatenating all training data
! cat $TOKENIZED_PATH_AYM_ES/train.aym $TOKENIZED_PATH_AYM_ES/train.es $TOKENIZED_PATH_ES_EN/train.es $TOKENIZED_PATH_ES_EN/train.en > $BIN_DIR/train.all

In [21]:
! fairseq-preprocess --source-lang all \
    --trainpref $BIN_DIR/train \
    --destdir $BIN_DIR \
    --workers 20 \
    --only-source

2022-09-20 13:09:12 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=

In [22]:
! fairseq-preprocess --source-lang aym --target-lang es \
    --trainpref $TOKENIZED_PATH_AYM_ES/train --validpref $TOKENIZED_PATH_AYM_ES/dev \
    --destdir $BIN_DIR \
    --srcdict $BIN_DIR/dict.all.txt \
    --tgtdict $BIN_DIR/dict.all.txt \
    --workers 20

2022-09-20 13:09:30 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging

### Reusing es dict from quy-es preprocessing

In [23]:
! fairseq-preprocess --source-lang es --target-lang en \
    --trainpref $TOKENIZED_PATH_ES_EN/train --validpref $TOKENIZED_PATH_ES_EN/dev \
    --destdir $BIN_DIR \
    --srcdict $BIN_DIR/dict.all.txt \
    --tgtdict $BIN_DIR/dict.all.txt \
    --workers 20

2022-09-20 13:09:35 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/storage/master-thesis/models/resdrop__aym-es_es-en/tokenizers/aym-es-en/bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging

## Training model

In [24]:
cd $CODE_STORAGE

[Errno 2] No such file or directory: '$CODE_STORAGE'
/notebooks


In [25]:
! git clone https://github.com/dannigt/fairseq.git

fatal: destination path 'fairseq' already exists and is not an empty directory.


In [26]:
cd ..

/


In [27]:
! fairseq-train $BIN_DIR \
    --user-dir $CODE_STORAGE/fairseq/examples/residual_drop/residual_drop_src/ \
    --arch=residual_drop_transformer --share-all-embeddings \
    --task translation_multi_simple_epoch --lang-pairs aym-es,es-en \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --restore-file $MODEL_DIR/checkpoint_last.pt \
    --save-dir $MODEL_DIR/ \
    --keep-last-epochs 2 \
    --reset-optimizer \
    --encoder-langtok "src" \
    --decoder-langtok \
    --fp16 \
    --max-epoch 200 \
    --patience 10 \
    --encoder-drop-residual 2


2022-09-20 13:09:53 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': '/storage/code//fairseq/examples/residual_drop/residual_drop_src/', 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': Fal

In [ ]:
! echo $MODEL_DIR

/storage/master-thesis/models/quy-es+es-en


In [ ]:
cd /storage/master-thesis/models/quy-es+es-en

/storage/master-thesis/models/quy-es+es-en


In [ ]:
! rm checkpoint108.pt
! rm checkpoint109.pt
! rm checkpoint_best.pt
! rm checkpoint_last.pt

In [ ]:
! rm dict.en.txt
! rm dict.es.txt
! rm dict.quy.txt

In [ ]:
! du -shc /notebooks/*

2.5K	/notebooks/CITATION.cff
5.5K	/notebooks/CODE_OF_CONDUCT.md
15K	/notebooks/CONTRIBUTING.md
19K	/notebooks/ISSUES.md
12K	/notebooks/LICENSE
512	/notebooks/MANIFEST.in
3.5K	/notebooks/Makefile
41K	/notebooks/README.md
41K	/notebooks/README_zh-hans.md
42K	/notebooks/README_zh-hant.md
12K	/notebooks/docker
4.6M	/notebooks/docs
5.0M	/notebooks/examples
8.5K	/notebooks/hubconf.py
7.8G	/notebooks/master-thesis
1.5K	/notebooks/model_cards
9.5K	/notebooks/notebooks
512	/notebooks/pyproject.toml
64K	/notebooks/scripts
1.0K	/notebooks/setup.cfg
13K	/notebooks/setup.py
13M	/notebooks/src
731K	/notebooks/templates
4.5K	/notebooks/test quy-es -> es-en model.ipynb
6.8M	/notebooks/tests
147K	/notebooks/train es-en model.ipynb
158K	/notebooks/train quy-es + es-en model.ipynb
160K	/notebooks/utils
3.5K	/notebooks/valohai.yaml
7.9G	total


In [ ]:
! du -shc /notebooks/master-thesis/*

512	/notebooks/master-thesis/Untitled.ipynb
362M	/notebooks/master-thesis/corpora
7.5G	/notebooks/master-thesis/models
7.8G	total


In [ ]:
! du -shc /notebooks/master-thesis/models/*

1.6G	/notebooks/master-thesis/models/es-en
802M	/notebooks/master-thesis/models/quy-es
5.2G	/notebooks/master-thesis/models/quy-es+es-en
7.5G	total


In [ ]:
! du -shc /*

19M	/apex
5.0M	/bin
4.0K	/boot
24K	/content
^C


In [ ]:
ls

 CITATION.cff         docker/          setup.py
 CODE_OF_CONDUCT.md   docs/            src/
 CONTRIBUTING.md      examples/        templates/
 ISSUES.md            hubconf.py      'test quy-es -> es-en model.ipynb'
 LICENSE              master-thesis/   tests/
 MANIFEST.in          model_cards/    'train es-en model.ipynb'
 Makefile             notebooks/      'train quy-es + es-en model.ipynb'
 README.md            pyproject.toml   utils/
 README_zh-hans.md    scripts/         valohai.yaml
 README_zh-hant.md    setup.cfg


/storage/master-thesis
